## 단순 홀드아웃 검증
>  #### 데이터의 일정량을 테스트 세트로 떼어 놓는다. 남은 데이터에서 훈련하고 테스트 세트로 평가한다. 정보 누설을 막기 위해 테스트 세트를 사용하여 모델을 튜닝해서는 안되기에 검증세트도 따로 떼어 놓아야 한다.
> 근데 솔직히 사이킷런의 train_test_split()이 굉장히 편하다.

In [ ]:
num_validation_samples = 10000

np.random.shuffle(data) # 데이터를 셔플링으로 섞어준다.

validation_data = data[:num_validation_samples] # 검증 세트를 만든다.
data = data[num_validation_samples:]

training_data = data[:] # 훈련세트를 만든다.(리스트 전체)

model = get_model()
model.train(training_data)
validation_score = model.evaluate(validation_data) # 훈련세트에서 모델을 훈련하고 검증세트로 평가한다.

# 모델을 튜닝
#다시 훈련, 평가, 튜닝을 반복

model = get_model()
model.train(np.concatenate([training_data, validation_data])) # 최적의 하이퍼파라미터를 구한후 마지막 모델 훈련을 시킬때 데이터를 모두 사용하는 것(테스트 빼고)
test_score = model.evaluate(test_data)

>  #### 하지만 너무 단순해서 한가지 단점이 있는데, 데이터가 적으면 주어진 전체 데이터를 통계적으로 대표하지 못할 수 있다는 점이다. 다른 난수 초깃값으로 셔플링해서 데이터를 나누었을 때 모델의 성능이 매우 달라질 수 있다.

## K-겹 교차검증
> #### 이 방식에서는 데이터를 동일한 크기를 가진 K개 분할으로 나눈다.
> 각 분할 i에 대해 남은 K-1개의 분할로 모델을 훈련하고 분할 i에서 모델을 평가한다.
> 최종점수는 이렇게 얻은 K개의 점수를 평균한다.
---------------------------------------
> #### 이 방법은 모델의 성능이 데이터 분할에 따라 편차가 클 때 도움이 된다.

In [ ]:
K = 4
num_validation_samples = len(data) // k

np.random.shuffle(data)

validation_score = []
for fold in range(k):
    validation_data = data[num_validation_samples * fold:
     num_validation_samples * (fold + 1)] # 검증 데이터 부분을 선택한다.
    training_data = data[:num_validation_samples * fold] + 
    # 남은 데이터를 훈련데이터로 사용하기 때문에 +연산자로 두 리스트를 연결한다.
    data[num_validation_samples * (fold + 1):]
    
    model = get_model() # 훈련되지 않은 새로운 모델 만들기
    model.train(training_data)
    validation_score = model.evaluate(validation_data)
    validation_score,append(validation_score)
    
validation_score = np.average(validation_score) # 검증 점수 : K개 퐁드의 검증 점수 평균

model = get_model()
model.train(data)
test_score = model.evaluate(test_data) # 테스트 데이터를 제외한 전체 데이터로 최종 모델 훈련

> K-겹 교차검증은 사이킷런의 cross_validate()함수를 사용하여 쉽게 구현할 수 있다. 케라스와 사용하려면 KerasClassifier나 KerasRegresser 클래스로 모델을 감싸야 한다.

## 셔플링을 사용한 K-겹 교차검증
> #### 비교적 가용데이터가 크고 가능한 정확하게 모델을 평가하고자 할 때 사용한다. 캐글에서 이 방법이 아주 크게 도움이 된다고 한다!! 이 방법은 K-겹 교차검증을 여러번 적용되 K개의 분할로 나누기 전에 매번 데이터를 무작위로 섞고 시작한다고 한다. 그래서 비용이 매우 많이 든다.

> 반복 K-겹 교차검증은 RepeatedKFold 와 RepeatedStratifiedKFold 클래스를 cross_validate() 함수에 적용하여 구현할 수 있다고 한다.